In [121]:
"""
Blasts genes from svevo and zavitan and finds putative orthologues
Outputs first, a table with genes corresponding to each reference
Adds interesting information to output, position in genome according to annotation,
position of blast against genome, GO:terms
"""
#blastn -perc_identity 85 -subject disk/counts/svevo.counts.fa -query disk/counts/zav_v2.counts.fa -outfmt "6 qseqid sseqid pident mismatch gapopen qstart qend sstart send evalue length qlen slen"  -evalue 10e-10 > disk/counts/blast_zv_sv.csv
#blastn -perc_identity 85 -query disk/counts/svevo.counts.fa -subject disk/counts/zav_v2.counts.fa -outfmt "6 qseqid sseqid pident mismatch gapopen qstart qend sstart send evalue length qlen slen"  -evalue 10e-10 > disk/counts/blast_sv_zv.csv

'\nBlasts genes from svevo and zavitan and finds putative orthologues\nOutputs first, a table with genes corresponding to each reference\nAdds interesting information to output, position in genome according to annotation,\nposition of blast against genome, GO:terms\n'

In [122]:
import pandas as pd

In [123]:
file_zavitan = 'disk/counts/zavitan_v2.genes.csv'
file_svevo = 'disk/counts/svevo.genes.csv'
file_blast = 'disk/counts/blast_sv_zv.csv'

df_zavitan = pd.read_csv(file_zavitan, sep=',', )
df_svevo = pd.read_csv(file_svevo, sep=',', )

df_blast = pd.read_csv(file_blast, sep='\t', header=None)
df_blast.columns = ['svevo','zavitan','pident','mismatch','gapopen','qstart','qend','sstart','send','evalue','length','qlen','slen']


In [124]:
df_blast = df_blast[((df_blast.length / df_blast.qlen) >= 0.7) &
       ((df_blast.length / df_blast.slen) >= 0.7) &
       ((df_blast.length / df_blast.qlen) <= 1.3) &
       ((df_blast.length / df_blast.slen) <= 1.3)]

print('blast:',len(df_blast_svevo.index))
df_blast.head(4)

blast: 195


,svevo,zavitan,pident,mismatch,gapopen,qstart,qend,sstart,send,evalue,length,qlen,slen
2,TRITD3Av1G027010,TRIDC3Av2G023650,99.859,4,0,1,2839,1,2839,0.0,2839,2839,2839
3,TRITD3Av1G027040,TRIDC3Av2G023690,100.000,0,0,1162,4852,39,3729,0.0,3691,4852,3729
10,TRITD3Av1G027200,TRIDC3Av2G023850,97.690,114,18,2078,10540,2294,10718,0.0,8485,10540,10718
13,TRITD3Av1G027230,TRIDC3Av2G023890,99.538,46,7,1,13633,1,13632,0.0,13641,13633,13632


In [125]:
df_zavitan = df_zavitan[['id']]
df_svevo = df_svevo[['id']]

In [126]:
df_zavitan_merge = pd.merge(df_zavitan, df_blast, left_on='id', right_on='zavitan', how='left')
df_zavitan_merge = df_zavitan_merge[['id','svevo']]
df_zavitan_merge.head(3)

,id,svevo
0,TRIDC3Av2G023650,TRITD3Av1G027010
1,TRIDC3Av2G023690,TRITD3Av1G027040
2,TRIDC3Av2G023850,TRITD3Av1G027200


In [127]:
df_svevo_merge = pd.merge(df_svevo, df_blast, left_on='id', right_on='svevo', how='left')
df_svevo_merge = df_svevo_merge[['id','zavitan']]
df_svevo_merge.head(3)

,id,zavitan
0,TRITD3Av1G026840,NaN
1,TRITD3Av1G026860,NaN
2,TRITD3Av1G027010,TRIDC3Av2G023650


In [128]:
df_merge = pd.merge(df_svevo_merge, df_zavitan_merge, left_on='id', right_on='svevo', how='outer')
df_merge = df_merge[['id_x','id_y']]
df_merge.columns = ['svevo','zavitan']


In [129]:
df_merge.to_csv('disk/counts/cross.csv', sep=',', index=None)

In [180]:
print(len(df_merge.index))
df_merge.head(3)

297


,svevo,zavitan
0,TRITD3Av1G026840,NaN
1,TRITD3Av1G026860,NaN
2,TRITD3Av1G027010,TRIDC3Av2G023650


In [168]:
#add info to list
file_ann_svevo = 'disk/PGSB_TRITD_Jan2017_all.gff3'
file_ann_zav_v2 = 'disk/SORTED_WEW_v2_HC_e_LC_GFF3_CATTATI_gff_PAO_26feb18.gff3'

df_ann_sv = pd.read_csv(file_ann_svevo, index_col=False, sep='\t', header=None)
df_ann_sv.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

df_ann_zv = pd.read_csv(file_ann_zav_v2, index_col=False, sep='\t', header=None)
df_ann_zv.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

df_ann_zv = df_ann_zv[df_ann_zv.feature=='gene']
df_ann_zv = df_ann_zv[df_ann_zv.seqname == 'chr3A']
df_ann_sv = df_ann_sv[df_ann_sv.feature =='gene']
df_ann_sv = df_ann_sv[df_ann_sv.seqname == 'chr3A']



In [169]:
df_ann_sv['gene'] = df_ann_sv.attribute.str.split(';', expand=True)[0].str.split('=', expand=True)[1]
df_ann_zv['gene'] = df_ann_zv.attribute.str.split(';', expand=True)[0].str.split('=', expand=True)[1]

df_ann_sv = df_ann_sv[['seqname','start','end','gene']]
df_ann_zv = df_ann_zv[['seqname','start','end','gene']]

In [185]:
#adds svevo coordinates
df_merge_svevo = pd.merge(df_merge, df_ann_sv, left_on='svevo', right_on='gene', how='left')
df_merge_svevo = df_merge_svevo[['seqname','start','end','svevo','zavitan']]
df_merge_svevo.columns = ['svevo.seqname','svevo.start','svevo.end','svevo.gene','zavitan.gene']

In [186]:
df_merge_info.head(5)

,svevo.seqname,svevo.start,svevo.end,svevo.gene,zavitan.gene
0,chr3A,54132399,54132875,TRITD3Av1G026840,NaN
1,chr3A,54184665,54187934,TRITD3Av1G026860,NaN
2,chr3A,54339905,54342744,TRITD3Av1G027010,TRIDC3Av2G023650
3,chr3A,54405290,54410142,TRITD3Av1G027040,TRIDC3Av2G023690
4,chr3A,54889963,54900503,TRITD3Av1G027200,TRIDC3Av2G023850


In [187]:
#adds zavitan coordinates
df_merge_zavitan = pd.merge(df_merge_svevo, df_ann_zv, left_on='zavitan.gene', right_on='gene', how='left')
df_merge_zavitan = df_merge_zavitan[['svevo.seqname','svevo.start','svevo.end','svevo.gene','zavitan.gene','seqname','start','end']]
df_merge_zavitan.columns = ['svevo.seqname','svevo.start','svevo.end','svevo.gene','zavitan.gene','zavitan.seqname','zavitan.start','zavitan.end']



In [188]:
df_merge_zavitan.to_csv('disk/counts/cross_info.csv', sep=',', index=None)